In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
import math

In [135]:
tickers = ["NVDA", "NFLX", "AMC", "RIVN", "PLTR", "AMZN", "TSLA"]
etfs = ["SPY", "IWM", "DIA"]

class DataDownloader:
    def __init__(self, tickers, etfs):
        self.tickers = tickers
        self.etfs = etfs

    def download(self, period='1y'):
        return yf.download(self.tickers + self.etfs, period=period)

    def quarterly(self):
        return self.download(period='3mo')

    def annual(self):
        return self.download(period='1y')
    
    def decade(self):
        return self.download(period='10y')

# Usage:
downloader = DataDownloader(tickers, etfs)
pre_quarterly_data = downloader.quarterly()
pre_annual_data = downloader.annual()
pre_decade_data = downloader.decade()

[*********************100%%**********************]  10 of 10 completed
[*********************100%%**********************]  10 of 10 completed
[*********************100%%**********************]  10 of 10 completed


In [136]:
# I have this cell so that when I want to call back to the default df i can use prefix(pre_) for future use.
quarterly_data= pre_quarterly_data['Adj Close']
annual_data= pre_annual_data['Adj Close']
decade_data= pre_decade_data['Adj Close']

In [99]:
annual_data.tail()

,AMC,AMZN,DIA,IWM,NFLX,NVDA,PLTR,RIVN,SPY,TSLA
Date,,,,,,,,,,
2023-09-11,7.34,143.100006,346.453766,184.360001,445.359985,451.779999,15.79,23.410000,446.873749,273.579987
2023-09-12,7.57,141.229996,346.294189,184.179993,434.690002,448.700012,15.59,23.580000,444.422363,267.480011
2023-09-13,8.24,144.850006,345.635925,182.970001,412.239990,454.850006,15.60,23.250000,444.940552,271.299988
2023-09-14,8.14,144.720001,349.016998,185.550003,400.489990,455.809998,15.83,24.110001,448.777008,276.040009
2023-09-15,8.36,140.389999,346.140015,183.610001,396.940002,439.000000,15.33,24.180000,443.369995,274.390015


# Part 1: We'll create an index with our Tickers:
           


In [5]:
table = pd.DataFrame(index=tickers)
table

""
NVDA
NFLX
AMC
RIVN
PLTR
AMZN
TSLA


# Part 2: We'll calculate weight using the following formula:
           
###   Price / Total Price

In [6]:
# Taking the last row of quartely_data.tail(), for most current price. Then summed to calculate portfolio value...
# assuming that each stock is bought only 1 time.
current_prices = quarterly_data[tickers].iloc[-1]
summed_prices = current_prices.sum()


#Each value in the row 'current_price' will be calculated to append it's weight % to a new column 
weight_list = []
for price in current_prices:
    weight = (price/summed_prices) * 100
    weight_list.append(str(round(weight,2)) + " lbs")

table['Weight'] = weight_list
table

,Weight
NVDA,33.81 lbs
NFLX,30.57 lbs
AMC,0.64 lbs
RIVN,1.86 lbs
PLTR,1.18 lbs
AMZN,10.81 lbs
TSLA,21.13 lbs


# Part 3: We'll calculate 3-month trailing Annualized Volatility using the following formula:
           
### ( STD [ Daily Return ] )( SQRT [ Trading Days] ) * 100

In [7]:
def calculate_annualized_volatility(ticker_data):
    daily_return = ticker_data.pct_change().dropna()
    std_dev = daily_return.std()
    trading_days = daily_return.count()
    annualized_volatility = std_dev * math.sqrt(trading_days) * 100
    return annualized_volatility 

# avs- annualized volatilities 
avs = {}
for ticker in tickers:
    avs[ticker] = calculate_annualized_volatility(quarterly_data[ticker])
    
#Create Annualized votality column
for stock, av in avs.items():    
    table.loc[stock, 'Annualized Volatility'] = (str(round(av,2)) + "%")

In [8]:
table

,Weight,Annualized Volatility
NVDA,33.81 lbs,20.12%
NFLX,30.57 lbs,17.67%
AMC,0.64 lbs,85.62%
RIVN,1.86 lbs,36.29%
PLTR,1.18 lbs,32.48%
AMZN,10.81 lbs,15.23%
TSLA,21.13 lbs,26.9%


In [9]:
annual_data

,AMC,AMZN,DIA,IWM,NFLX,NVDA,PLTR,RIVN,SPY,TSLA
Date,,,,,,,,,,
2022-09-16,89.800003,123.529999,301.989807,176.111740,240.130005,131.899323,7.78,39.259998,379.584503,303.350006
2022-09-19,91.800003,124.660004,304.048523,177.617126,243.630005,133.738220,7.86,37.939999,382.528168,309.070007
2022-09-20,87.099998,122.190002,300.980133,175.196701,242.850006,131.679459,7.62,35.750000,378.137299,308.730011
2022-09-21,86.000000,118.540001,295.872589,172.520416,236.869995,132.528931,7.69,35.099998,371.541138,300.799988
2022-09-22,78.500000,117.309998,294.608002,168.633957,237.050003,125.533218,7.37,33.330002,368.420258,288.589996
...,...,...,...,...,...,...,...,...,...,...
2023-09-11,7.340000,143.100006,346.453766,184.360001,445.359985,451.779999,15.79,23.410000,446.873749,273.579987
2023-09-12,7.570000,141.229996,346.294189,184.179993,434.690002,448.700012,15.59,23.580000,444.422363,267.480011
2023-09-13,8.240000,144.850006,345.635925,182.970001,412.239990,454.850006,15.60,23.250000,444.940552,271.299988


# Part 4-6: 12-month trailing beta agaisnt's etfs:
           


# https://www.investopedia.com/terms/b/beta.asp

## Beta is equal to the :
### Covariance divided by variance.
#### Covariance = (returns on individual stocks agaisnt returns of ETF's).cov
#### Variance = (returns on ETF's).var

In [10]:
# First define daily returns
returns = annual_data.pct_change().dropna()

# Function to calculate beta
def calculate_beta(stock, etf):
    covariance = returns[stock].cov(returns[etf])
    variance = returns[etf].var()
    beta = covariance / variance
    return beta

# Calculate beta for each stock against each ETF and append to table
for etf in etfs:
    beta_column = []
    for stock in tickers:
        beta = calculate_beta(stock, etf)
        beta_column.append(round(beta,2))
    table[f'Beta against {etf}'] = beta_column


In [101]:
table

,Weight,Annualized Volatility,Beta against SPY,Beta against IWM,Beta against DIA,Average Weekly Drawdown,Maximum Weekly Drawdown
NVDA,33.81 lbs,20.12%,2.08,1.29,1.80,-177.35,-132.94
NFLX,30.57 lbs,17.67%,1.46,0.99,1.43,-177.35,-165.93
AMC,0.64 lbs,85.62%,1.80,1.68,1.82,-177.35,-211.68
RIVN,1.86 lbs,36.29%,2.34,1.86,2.23,-177.35,-205.29
PLTR,1.18 lbs,32.48%,1.85,1.45,1.62,-177.35,-149.07
AMZN,10.81 lbs,15.23%,1.50,0.98,1.32,-177.35,-169.24
TSLA,21.13 lbs,26.9%,1.78,1.31,1.44,-177.35,-211.29


# Part 7-8:
           

In [127]:

# Using pre_annual because annual_data wont work as it uses [Adj Close], not [High] & [Low]
highs = pre_annual_data['High'].pct_change().dropna()
lows = pre_annual_data['Low'].pct_change().dropna()
awd = []
for ticker in tickers:
    avg_highs = highs[ticker].mean()
    avg_lows = lows[ticker].mean()
    avg_weekly_drawdown = ((avg_lows - avg_highs) / avg_highs) * 100 
    awd.append(round(avg_weekly_drawdown, 2))

table['Average Weekly Drawdown'] = awd


In [132]:
mwd = []
for ticker in tickers:
    max_highs = highs[ticker].max()
    min_lows = lows[ticker].min()
    max_weekly_drawdown = ((min_lows - max_highs) / max_highs) * 100
    mwd.append(round(max_weekly_drawdown, 2))
    
table['Maximum Weekly Drawdown'] = mwd



In [133]:
table

,Weight,Annualized Volatility,Beta against SPY,Beta against IWM,Beta against DIA,Average Weekly Drawdown,Maximum Weekly Drawdown
NVDA,33.81 lbs,20.12%,2.08,1.29,1.80,-0.59,-128.46
NFLX,30.57 lbs,17.67%,1.46,0.99,1.43,5.13,-176.71
AMC,0.64 lbs,85.62%,1.80,1.68,1.82,13.09,-190.20
RIVN,1.86 lbs,36.29%,2.34,1.86,2.23,9.94,-190.40
PLTR,1.18 lbs,32.48%,1.85,1.45,1.62,-5.76,-142.53
AMZN,10.81 lbs,15.23%,1.50,0.98,1.32,1.64,-203.87
TSLA,21.13 lbs,26.9%,1.78,1.31,1.44,47.60,-205.75


# Part 9:

In [172]:
total_return_list = []
for ticker in tickers:
    current_value = decade_data[ticker].iloc[-1]
    initial_value = decade_data[ticker].dropna().iloc[0]
    difference = current_value - initial_value
    total_return = (difference / initial_value) * 100
    total_return_list.append(str(round(total_return,2)) + "%")

table['Total Return'] = total_return_list
table

,Weight,Annualized Volatility,Beta against SPY,Beta against IWM,Beta against DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return
NVDA,33.81 lbs,20.12%,2.08,1.29,1.80,-0.59,-128.46,11735.53%
NFLX,30.57 lbs,17.67%,1.46,0.99,1.43,5.13,-176.71,819.57%
AMC,0.64 lbs,85.62%,1.80,1.68,1.82,13.09,-190.20,-95.43%
RIVN,1.86 lbs,36.29%,2.34,1.86,2.23,9.94,-190.40,-76.0%
PLTR,1.18 lbs,32.48%,1.85,1.45,1.62,-5.76,-142.53,61.37%
AMZN,10.81 lbs,15.23%,1.50,0.98,1.32,1.64,-203.87,848.39%
TSLA,21.13 lbs,26.9%,1.78,1.31,1.44,47.60,-205.75,2370.79%


In [183]:
atl = []
for ticker in tickers:
    current_value = decade_data[ticker].iloc[-1]
    initial_value = decade_data[ticker].dropna().iloc[0]
    annualized_total_return = (((current_value / initial_value)**1/10)-1) * 100
    atl.append(str(round(annualized_total_return,2)) + "%")
table['Annualized Total Return'] = atl
table

,Weight,Annualized Volatility,Beta against SPY,Beta against IWM,Beta against DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return,Annualized Total Return
NVDA,33.81 lbs,20.12%,2.08,1.29,1.80,-0.59,-128.46,11735.53%,1083.55%
NFLX,30.57 lbs,17.67%,1.46,0.99,1.43,5.13,-176.71,819.57%,-8.04%
AMC,0.64 lbs,85.62%,1.80,1.68,1.82,13.09,-190.20,-95.43%,-99.54%
RIVN,1.86 lbs,36.29%,2.34,1.86,2.23,9.94,-190.40,-76.0%,-97.6%
PLTR,1.18 lbs,32.48%,1.85,1.45,1.62,-5.76,-142.53,61.37%,-83.86%
AMZN,10.81 lbs,15.23%,1.50,0.98,1.32,1.64,-203.87,848.39%,-5.16%
TSLA,21.13 lbs,26.9%,1.78,1.31,1.44,47.60,-205.75,2370.79%,147.08%
